In [1]:
# TODO: reorg
# simplify code
import json
import time
import uuid
from collections import defaultdict

from IPython.display import Javascript, display
from ipywidgets import (Box, Button, Checkbox, ColorPicker, Dropdown,
                        FloatSlider, FloatText, HBox, IntSlider, IntText,
                        Label, Layout, Play, Tab, Text, Textarea, ToggleButton,
                        VBox, interactive, jslink, AppLayout, GridspecLayout)
from traitlets import (Any, Bool, CaselessStrEnum, Dict, Float, HasTraits, Int,
                       link, observe)
import traitlets

# import default

# from parameters import REPRESENTATION_NAMES
# from utils import js_utils


In [2]:
# TODO: reorg
# simplify code
from ipywidgets import Box, Button, Label, Layout, Tab, ToggleButton


def make_form_item_layout():
    # protect nglview if it is not used in notebook (wheen there is not `comm`
    return Layout(
        display='flex',
        flex_flow='row',
        justify_content='space-between',
    )


def _make_box_layout(width='100%'):
    return Layout(display='flex',
                  flex_flow='column',
                  align_items='stretch',
                  width=width)


def _relayout(box, form_item_layout):
    form_items = []
    for kid in box.children:
        if hasattr(kid,
                   'description') and not isinstance(kid,
                                                     (Button, ToggleButton)):
            label_value = kid.description
            kid.description = ''
        else:
            label_value = ''
        if isinstance(kid, Button):
            box2 = Box([
                kid,
            ], layout=form_item_layout)
        else:
            box2 = Box([Label(value=label_value), kid],
                       layout=form_item_layout)
        form_items.append(box2)

    return form_items


def _relayout_master(box, width='20%'):
    """make nicer layout for box.

    This method will take the `description` of each child to make corresponding Label
    The `description` will be cleared.
    """
    old_children = box.children[:]
    form_items = _relayout(box, make_form_item_layout())
    form = Box(form_items, layout=_make_box_layout(width=width))
    form._ngl_children = old_children
    return form


def _make_autofit(box):
    '''

    Parameters
    ----------
    box : ipywidgets.Box
        children is a list of buttons

    Returns
    -------
    relayouted box
    '''
    items_layout = Layout(flex='1 1 auto', width='auto')

    box.layout = items_layout
    return box


def _make_delay_tab(box_factory, selected_index=0):
    """

    Parameters
    ----------
    box_factory : list of (func, tab_name)

    Example of box_factory: [(_make_gen_box, 'General'),
                             (_make_repr_box, 'Representation')]
    """

    tab = Tab([Box() for box, _ in box_factory])
    [tab.set_title(i, title) for i, (_, title) in enumerate(box_factory)]

    # trick
    if not tab.children[selected_index].children:
        tab.selected_index = 1

    def on_update_selected_index(change):
        index = change['new']
        if not tab.children[index].children:
            # make widget on demand
            tab.children[index].children = [
                box_factory[index][0](),
            ]

    tab.observe(on_update_selected_index, names='selected_index')

    # trigger
    tab.selected_index = selected_index

    return tab


In [4]:
class TrajectoryRSCB(HasTraits):
    # should set default values here different from desired defaults
    # so `observe` can be triggered
    widget_structure_view = Any(None)
    widget_ligand_view = Any(None)
    widget_electron_density_maps = Any(None)
    widget_tab = Any(None)
    app = Any(None)

    assembly_type = CaselessStrEnum(['Asymmetric Unit', 'Bioassembly 1',
        'Unitcell', 'Supercell'], default_value='Asymmetric Unit')

    model_type = CaselessStrEnum(['Model 1'], default_value='Model 1')

    symmetry_type = CaselessStrEnum(['None'], default_value='None')

    style_type = CaselessStrEnum(['None', 'Backbone', 'Surface', 'Cartoon',
        'Spacefill', 'Licorice', 'Line'], default_value='None')

    color_type = CaselessStrEnum(['None','By Chain', 'Rainbow',
        'By Element/CPI', 'By Residue','By Molecule Type', 'By B-Factor',
        'By Secondary Structure','By Hydrophobic', 'By Density Fit',
        'By Geometry Quantity'], default_value='None')

    ligand_type = CaselessStrEnum(['None', 'Ball & Stick', 'Spacefill'], default_value='None')
    
    def __init__(self, view):
        self._view = view

    def _make_structure_view_box(self):

        if self.widget_structure_view is None:
            _widget_assembly = Dropdown(value='Asymmetric Unit',
                options=['Asymmetric Unit', 'Bioassembly 1', 'Unitcell', 'Supercell'],
                description='Assembly')
            _widget_model = Dropdown(value='Model 1',
                options=['Model 1'],
                description='Model')
            _widget_symmetry = Dropdown(value='None',
                options=['None'],
                description='Symmetry')
            _widget_style = Dropdown(value='None',
                options=['None', 'Backbone', 'Surface', 'Cartoon',
                            'Spacefill', 'Licorice', 'Line'],
                description='Style')
            _widget_color = Dropdown(value='By Chain',
                options=['None','By Chain', 'Rainbow', 'By Element/CPI', 'By Residue',
                                 'By Molecule Type', 'By B-Factor',
                                 'By Secondary Structure',
                                 'By Hydrophobic',
                                 'By Density Fit',
                                 'By Geometry Quantity'],
                description='Color')
            _widget_ligand = Dropdown(value='None',
                options=['None', 'Ball & Stick', 'Spacefill'],
                description='Ligand')

            checkbox_water = Checkbox(value=False, description='Water', width='20px')
            checkbox_ions = Checkbox(value=True, description='Ions', width='20px')
            checkbox_hydrogens = Checkbox(value=True, description='Hydrogens', width='20px')
            checkbox_clashes = Checkbox(value=False, description='Clashes', width='20px')
            #checkbox_water = set_max_width_check_box(checkbox_water)
            #checkbox_ions = set_max_width_check_box(checkbox_ions)
            #checkbox_hydrogens = set_max_width_check_box(checkbox_hydrogens)
            #checkbox_clashes = set_max_width_check_box(checkbox_clashes)

            link((_widget_assembly, 'value'), (self, 'assembly_type'))
            link((_widget_model, 'value'), (self, 'model_type'))
            link((_widget_symmetry, 'value'), (self, 'symmetry_type'))
            link((_widget_style, 'value'), (self, 'style_type'))
            link((_widget_color, 'value'), (self, 'color_type'))
            link((_widget_ligand, 'value'), (self, 'ligand_type'))

            default_button_str_view = Button(description='Default Structure View')

            @default_button_str_view.on_click
            def on_click(default_button_str_view):
                # TO DO FIX THIS PARAMETER PART
                pass
            form_item_layout = Layout(
                flex_flow='row',
                flex='1 1 auto',
                width='auto')
            
            box_layout=Layout(
                display='flex',
                flex_flow='column',
                align_items='stretch',
                width='auto')
            
            center_hbox_water_ions = VBox([
                    checkbox_water,
                    checkbox_ions
                ], layout=form_item_layout)

            center_hbox_hydrogens_clashes = VBox([
                    checkbox_hydrogens,
                    checkbox_clashes
                ], layout=form_item_layout)


            vbox_view = Box([
                _widget_assembly,
                _widget_model,
                _widget_symmetry,
                _widget_style,
                _widget_color,
                _widget_ligand,
                center_hbox_water_ions,
                center_hbox_hydrogens_clashes,
                default_button_str_view
                ], layout=box_layout)

            self.widget_structure_view = vbox_view 
        return self.widget_structure_view

    @observe('assembly_type')
    def update_assembly_type(self, change):
        self.asymetric_type = change['new']

    @observe('model_type')
    def update_model_type(self, change):
        self.model_type = change['new']

    @observe('symmetry_type')
    def update_symmetry_type(self, change):
        self.symmetry_type = change['new']

    @observe('style_type')
    def update_style_type(self, change):
        self.style_type = change['new']

    @observe('color_type')
    def update_color_type(self, change):
        self.color_type = change['new']

    @observe('ligand_type')
    def update_ligand_type(self, change):
        self.ligand_type = _ligand_type = change['new']
        self._view.clear()
        if _ligand_type == 'Ball & Stick':
            self._view.add_ball_and_stick('ligand')
        elif _ligand_type == 'Spacefill':
            self._view.add_spacefill('ligand')
    def _display(self):
        if self.app is None:
            self.widget_structure_view = self._make_structure_view_box()
            self.app = AppLayout(header=None, 
                                 center=self._view, 
                                 right_sidebar=self.widget_structure_view, 
                                 left_sidebar=None,
                                 pane_widths = [0, 3, 2],
                                 justify_items='center',
                                 aligh_items='center'
                                )
            self.app.width = '100%'
            #self.app.display = 'inline-flex'
        return self.app

    

    

In [5]:
import nglview as nv
view = nv.demo()
test_1 = TrajectoryRSCB(view)
test_1._display()


AppLayout(children=(Box(children=(Dropdown(description='Assembly', options=('Asymmetric Unit', 'Bioassembly 1'…